In [67]:
from collections import defaultdict, OrderedDict
from typing import List
import heapq

**706. Design HashMap**

In [15]:
class Node:
    def __init__(self, key: int, val: int, next=None):
        self.key = key
        self.val = val
        self.next = next


class MyHashMap:
    def __init__(self):
        self.mapLength = 50
        self.map = [Node(-1, -1) for _ in range(self.mapLength)]

    def put(self, key: int, value: int) -> None:
        hashKey = hash(key) % self.mapLength
        cur = self.map[hashKey]
        while cur.next and cur.key != key:
            cur = cur.next
        if cur.key == key:
            cur.val = value
        else:
            cur.next = Node(key, value)

    def get(self, key: int):
        hashKey = hash(key) % self.mapLength
        cur = self.map[hashKey]
        while cur.next and cur.key != key:
            cur = cur.next
        if cur.key == key:
            return cur.val
        return -1

    def remove(self, key: int):
        hashKey = hash(key) % self.mapLength
        cur = self.map[hashKey]
        while cur.next and cur.next.key != key:
            cur = cur.next

        if not cur.next:
            return

        cur.next = cur.next.next


Map = MyHashMap()
Map.put(9, 10)
Map.put(59, 11)

Map.remove(59)
Map.get(59)

-1

**1396. Design underground system**


- store a hashmap whose keys are station names
    - the value of the entries will be (number of trips, averageTime)
- store a hashmap whose keys are user IDs
    - the value of the entries will be (current checkIn station if available, checkIn time if available)
    - these will get cleared when the user checks out

This system works because average time is computed only from **direct** trips, if it was any trip that started and ended at the two given stations, these data structures would not work.

In [ ]:
class UndergroundSystem:  # 72% time, 53% memory
    def __init__(self):
        self.stationTrips = defaultdict(
            dict
        )  # station -> {destination station: [number of trips, average number of trips]}
        self.userTrips = defaultdict(
            list
        )  # user -> [check in station | None, check in time | None]

    def checkIn(self, id: int, stationName: str, t: int) -> None:
        self.userTrips[id] = [stationName, t]

    def checkOut(self, id: int, stationName: str, t: int) -> None:
        checkInStation, checkInTime = self.userTrips[id]
        # checking if the check out station already has data in the check in station's history
        if stationName in self.stationTrips[checkInStation]:
            numTrips, averageTime = self.stationTrips[checkInStation][stationName]
        else:
            numTrips, averageTime = [0, 0]

        newAverageTime = (averageTime * numTrips + (t - checkInTime)) / (numTrips + 1)
        # modifying the average time for that specific trip
        self.stationTrips[checkInStation][stationName] = [numTrips + 1, newAverageTime]

    def getAverageTime(self, startStation: str, endStation: str) -> float:
        # at least on customer will have made this travel before this is ran
        # so there's no need to check if the data exists
        return self.stationTrips[startStation][endStation][1]


class UndergroundSystemAverageLater:  # 72% time, 53% memory
    def __init__(self):
        # station -> {destination station: [number of trips, total travel time from start to destination]}
        self.stationTrips = defaultdict(dict)
        # user -> [check in station, check in time]
        self.userTrips = defaultdict(list)

    def checkIn(self, id: int, stationName: str, t: int) -> None:
        self.userTrips[id] = [stationName, t]

    def checkOut(self, id: int, stationName: str, t: int) -> None:
        checkInStation, checkInTime = self.userTrips[id]
        # checking if the check out station already has data in the check in station's history
        if stationName not in self.stationTrips[checkInStation]:
            self.stationTrips[checkInStation][stationName] = [
                0,
                0,
            ]  # num trips, total time

        self.stationTrips[checkInStation][stationName][0] += (
            1  # will be used to calc average time in `getAverageTime`
        )
        self.stationTrips[checkInStation][stationName][1] += (
            t - checkInTime
        )  # added to total travel time

    def getAverageTime(self, startStation: str, endStation: str) -> float:
        # at least on customer will have made this travel before this is ran
        # so there's no need to check if the data exists
        totalTime = self.stationTrips[startStation][endStation][1]
        numTrips = self.stationTrips[startStation][endStation][0]
        return totalTime / numTrips

**355. Design Twitter**

In [ ]:
class Twitter:
    # 57% time, 60% memory
    def __init__(self):
        self.numPosts = 0
        self.tweetMap = defaultdict(list)  # userID -> list of [order posted, tweetID]
        self.followMap = defaultdict(set)  # userID -> set of following IDs

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.tweetMap[userId].append([self.numPosts, tweetId])
        self.numPosts -= 1  # decrementing so we can use a maxHeap

    def getNewsFeed(self, userId: int) -> List[int]:
        result = []
        minHeap = []

        # one time thing: has to be a follower of themself
        if userId not in self.followMap[userId]:
            self.followMap[userId].add(userId)

        for followeeId in self.followMap[userId]:
            if followeeId in self.tweetMap:
                index = len(self.tweetMap[followeeId]) - 1
                # index of the last tweet in that tweet array
                count, tweetId = self.tweetMap[followeeId][index]
                minHeap.append([count, tweetId, followeeId, index - 1])
                # index - 1 is the index of the next tweet in the tweet array for that followeeId

        heapq.heapify(minHeap)

        while minHeap and len(result) < 10:
            count, tweetId, followeeId, index = heapq.heappop(minHeap)
            result.append(tweetId)
            if index >= 0:  # equal because index 0 has a tweet as well
                count, tweetId = self.tweetMap[followeeId][index]
                heapq.heappush(minHeap, [count, tweetId, followeeId, index - 1])
        return result

    def follow(self, followerId: int, followeeId: int) -> None:
        self.followMap[followerId].add(followeeId)

    def unfollow(self, followerId: int, followeeId: int) -> None:
        try:
            self.followMap[followerId].remove(followeeId)
        except KeyError:
            pass


# Your Twitter object will be instantiated and called as such:
# obj = Twitter()
# obj.postTweet(userId,tweetId)
# param_2 = obj.getNewsFeed(userId)
# obj.follow(followerId,followeeId)
# obj.unfollow(followerId,followeeId)

**1603. Design Parking System**

In [ ]:
class ParkingSystem:
    def __init__(self, big: int, medium: int, small: int):
        # can be expanded to cars leaving
        # type -> [total, unnoccupied]
        self.parking = {1: [big, big], 2: [medium, medium], 3: [small, small]}

    def addCar(self, carType: int) -> bool:
        if self.parking[carType][1]:
            self.parking[carType][1] -= 1
            return True
        return False

**1472. Design Browser History**

In [44]:
class BrowserHistory:  # 76% time, 80% memory
    def __init__(self, homepage: str):
        self.history = [homepage]
        self.urlToIndex = {homepage: 0}

        self.curUrl = homepage
        # this allows me to not have a conditional check in back and forward
        # when validating the steps argument

    def visit(self, url: str) -> None:
        # truncating history if appropriate (the current url is not the last entry in self.history)
        i = len(self.history) - 1
        while i > self.urlToIndex[self.curUrl]:
            del self.urlToIndex[self.history[i]]
            self.history.pop()
            i -= 1

        self.history.append(url)
        self.curUrl = url
        self.urlToIndex[url] = len(self.history) - 1

    def back(self, steps: int) -> str:
        # if steps leads to a negative index, go to zero index
        backUrl = self.history[max(self.urlToIndex[self.curUrl] - steps, 0)]
        self.curUrl = backUrl
        return backUrl

    def forward(self, steps: int) -> str:
        # if steps leads to an index >= len(self.history), reset it to point to len(self.history) - 1
        forwardUrl = self.history[
            min(self.urlToIndex[self.curUrl] + steps, len(self.history) - 1)
        ]
        self.curUrl = forwardUrl
        return forwardUrl


class BrowserHistoryClever:
    def __init__(self, homepage: str):
        self.history = [homepage]
        self.pos = 0

    def visit(self, url: str) -> None:
        self.pos += 1
        if self.pos < len(self.history):
            self.history = self.history[: self.pos]
        self.history.append(url)

    def back(self, steps: int) -> str:
        self.pos = max(self.pos - steps, 0)
        return self.history[self.pos]

    def forward(self, steps: int) -> str:
        self.pos = min(self.pos + steps, len(self.history) - 1)
        return self.history[self.pos]


H = BrowserHistory("leetcode.com")
H.visit("google.com")
H.visit("facebook.com")
H.visit("youtube.com")
print(H.back(1))
print(H.back(1))
print(H.forward(1))
H.visit("linkedin.com")
print(H.forward(2))
print(H.back(7))
H.visit("twitch.tv")
display(H.history, H.urlToIndex, H.curUrl)

facebook.com
google.com
facebook.com
linkedin.com
leetcode.com


['leetcode.com', 'twitch.tv']

{'leetcode.com': 0, 'twitch.tv': 1}

'twitch.tv'

**1797. Design Authentication Manager**

In [66]:
class AuthenticationManager:  # 47% time, 82% memory
    def __init__(self, timeToLive: int):
        self.timeToLive = timeToLive
        self.tokens = {}  # tokenId -> expirationTime

    def generate(self, tokenId: str, currentTime: int) -> None:
        if tokenId not in self.tokens:
            self.tokens[tokenId] = currentTime + self.timeToLive

    def renew(self, tokenId: str, currentTime: int) -> None:
        if tokenId in self.tokens and self.tokens[tokenId] > currentTime:
            self.tokens[tokenId] = currentTime + self.timeToLive

    def countUnexpiredTokens(self, currentTime: int) -> int:
        unexpired = 0
        for token in self.tokens:
            if self.tokens[token] > currentTime:
                unexpired += 1
        return unexpired


class AuthenticationManagerOrderedDict:  # 93% time, 57% memory
    def __init__(self, timeToLive: int):
        self.timeToLive = timeToLive
        self.map = OrderedDict()

    def generate(self, tokenId: str, currentTime: int) -> None:
        expireTime = currentTime + self.timeToLive
        self.map[tokenId] = expireTime

    def renew(self, tokenId: str, currentTime: int) -> None:
        if tokenId not in self.map:
            return

        if self.map[tokenId] <= currentTime:
            return

        expireTime = currentTime + self.timeToLive
        self.map[tokenId] = expireTime
        self.map.move_to_end(tokenId)
        # move to the end of the order dictionary so that the expiration times are monotonically increasing
        # (so that they are sorted)

    def countUnexpiredTokens(self, currentTime: int) -> int:
        while self.map:
            # iter(self.map) returns an iterable over self.map's keys
            # next(...) then takes the first key available in the iterator
            # which is the oldest added key in the Ordered Dict
            # then we use that oldest added key to access its expiry time.
            # it won't duplicate the key retrieved because the oldest key is either removed or the loop breaks
            expTime = self.map[next(iter(self.map))]

            if expTime <= currentTime:
                self.map.popitem(last=False)
            else:  # since it is an ordered hashmap,
                # the moment a token is not expired, all tokens are no longer expired
                break
        return len(self.map)

        # while self.map and self.map[next(iter(self.map))] <= currentTime:
        #    self.map.popitem(last=False)

        # return len(self.map)